In [53]:
import pandas as pd
import numpy as np
import math
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from IPython.core.display import clear_output
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from tensorflow.python.keras import models
from tensorflow.python.keras import layers
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from sklearn.ensemble import RandomForestClassifier

In [3]:
# load balance sheet into df
balance_sheets = pd.read_csv("balance_sheets.csv")
balance_sheets.head()

,fiscalDateEnding,reportedCurrency,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,longTermDebtNoncurrent,shortLongTermDebtTotal,otherCurrentLiabilities,otherNonCurrentLiabilities,totalShareholderEquity,treasuryStock,retainedEarnings,commonStock,commonStockSharesOutstanding,Symbol
0,2021-09-30,USD,48278000000,16426000000,4878000000,4878000000,4950000000,4916000000,31852000000,9323000000,...,16193000000,18149000000,3242000000,3353000000,14460000000,29673000000,45361000000,9000000,576252803,MMM
1,2021-06-30,USD,48307000000,16343000000,4695000000,805000000,4842000000,4991000000,31995000000,9358000000,...,16267000000,18232000000,3181000000,3297000000,14448000000,29236000000,44824000000,9000000,578638253,MMM
2,2021-03-31,USD,47180000000,15345000000,4636000000,501000000,4458000000,4817000000,31866000000,9240000000,...,16819000000,18169000000,3126000000,3330000000,13760000000,29020000000,44255000000,9000000,579675002,MMM
3,2020-12-31,USD,47344000000,14982000000,4634000000,404000000,4239000000,4705000000,34027000000,9421000000,...,17989000000,18783000000,3278000000,3462000000,12867000000,29404000000,43821000000,9000000,577749638,MMM
4,2020-09-30,USD,45390000000,14110000000,4121000000,440000000,3984000000,4623000000,31314000000,9216000000,...,18429000000,19582000000,2840000000,3324000000,11880000000,29570000000,43285000000,9000000,576821878,MMM


In [4]:
# load cash flow into df
cash_flow = pd.read_csv("cash_flow.csv")
cash_flow.head()

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,...,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome,Symbol
0,2021-09-30,USD,1874000000,None,None,29000000,166000000,476000000,343000000,-13000000,...,None,None,None,None,-441000000,86000000,1437000000,None,1434000000,MMM
1,2021-06-30,USD,1887000000,None,None,73000000,472000000,472000000,394000000,132000000,...,None,None,None,None,-316000000,187000000,1525000000,29000000,1524000000,MMM
2,2021-03-31,USD,1688000000,None,None,197000000,509000000,460000000,310000000,205000000,...,None,None,None,None,62000000,293000000,2000000,-66000000,1624000000,MMM
3,2020-12-31,USD,2515000000,None,None,286000000,82000000,498000000,422000000,-52000000,...,None,None,None,None,102000000,104000000,1390000000,59000000,1389000000,MMM
4,2020-09-30,USD,2480000000,None,None,94000000,-113000000,481000000,368000000,128000000,...,None,None,None,None,89000000,89000000,1417000000,42000000,1430000000,MMM


In [5]:
# load earnings into df
earnings = pd.read_csv("earnings.csv")
earnings.head()

,fiscalDateEnding,reportedDate,reportedEPS,estimatedEPS,surprise,surprisePercentage,Symbol
0,2021-09-30,2021-10-26,2.45,2.2084,0.2416,10.94,MMM
1,2021-06-30,2021-07-27,2.59,2.2765,0.3135,13.7711,MMM
2,2021-03-31,2021-04-27,2.77,2.2932,0.4768,20.7919,MMM
3,2020-12-31,2021-01-26,2.38,2.1678,0.2122,9.7887,MMM
4,2020-09-30,2020-10-27,2.43,2.2669,0.1631,7.1948,MMM


In [6]:
# load income statements into df
income_statements = pd.read_csv("income_statements.csv")
income_statements.head()

,fiscalDateEnding,reportedCurrency,grossProfit,totalRevenue,costOfRevenue,costofGoodsAndServicesSold,operatingIncome,sellingGeneralAndAdministrative,researchAndDevelopment,operatingExpenses,...,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome,Symbol
0,2021-09-30,USD,4089000000,8942000000,4853000000,4853000000,1788000000,1819000000,482000000,1950000000,...,131000000,1758000000,324000000,117000000,1433000000,1300000000,1875000000,2006000000,1434000000,MMM
1,2021-06-30,USD,4231000000,8950000000,4719000000,4719000000,1971000000,1746000000,514000000,1880000000,...,134000000,1939000000,415000000,121000000,1523000000,1805000000,2060000000,2194000000,1524000000,MMM
2,2021-03-31,USD,4326000000,8851000000,4525000000,4525000000,1994000000,1808000000,524000000,1941000000,...,133000000,1943000000,319000000,132000000,1626000000,1578000000,2075000000,2208000000,1624000000,MMM
3,2020-12-31,USD,4195000000,8583000000,4388000000,4388000000,1849000000,1890000000,456000000,3189000000,...,153000000,1705000000,316000000,10000000,4185000000,1688000000,1846000000,1999000000,1389000000,MMM
4,2020-09-30,USD,4047000000,8350000000,4303000000,4303000000,1909000000,1677000000,461000000,1814000000,...,137000000,1800000000,391000000,128000000,1418000000,1745000000,1928000000,2065000000,1430000000,MMM


In [7]:
# load monthly prices into df
monthly_prices = pd.read_csv("monthly_prices.csv")
monthly_prices.columns = ['Date', 'open', 'high', 'low', 'close', 'volume', 'Symbol']
monthly_prices.head()

,Date,open,high,low,close,volume,Symbol
0,2022-01-21,178.32,181.780,172.5600,172.65,34039389,MMM
1,2021-12-31,172.55,179.180,169.1801,177.63,48251474,MMM
2,2021-11-30,178.53,186.300,169.9400,170.04,47872133,MMM
3,2021-10-29,176.25,183.820,173.7800,178.68,48241875,MMM
4,2021-09-30,194.55,195.606,175.3700,175.42,67366254,MMM


In [8]:
# merge all dataframes together
historical_data = pd.merge(balance_sheets, cash_flow, on = ['Symbol', 'fiscalDateEnding'])
historical_data = pd.merge(historical_data, earnings, on = ['Symbol', 'fiscalDateEnding'])
historical_data = pd.merge(historical_data, income_statements, on = ['Symbol', 'fiscalDateEnding'])
historical_data

,fiscalDateEnding,reportedCurrency_x,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,depreciation,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y
0,2021-09-30,USD,48278000000,16426000000,4878000000,4878000000,4950000000,4916000000,31852000000,9323000000,...,None,131000000,1758000000,324000000,117000000,1433000000,1300000000,1875000000,2006000000,1434000000
1,2021-06-30,USD,48307000000,16343000000,4695000000,805000000,4842000000,4991000000,31995000000,9358000000,...,None,134000000,1939000000,415000000,121000000,1523000000,1805000000,2060000000,2194000000,1524000000
2,2021-03-31,USD,47180000000,15345000000,4636000000,501000000,4458000000,4817000000,31866000000,9240000000,...,None,133000000,1943000000,319000000,132000000,1626000000,1578000000,2075000000,2208000000,1624000000
3,2020-12-31,USD,47344000000,14982000000,4634000000,404000000,4239000000,4705000000,34027000000,9421000000,...,None,153000000,1705000000,316000000,10000000,4185000000,1688000000,1846000000,1999000000,1389000000
4,2020-09-30,USD,45390000000,14110000000,4121000000,440000000,3984000000,4623000000,31314000000,9216000000,...,None,137000000,1800000000,391000000,128000000,1418000000,1745000000,1928000000,2065000000,1430000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9164,2017-12-31,USD,8586000000,4217000000,1564000000,1564000000,1427000000,1060000000,4289000000,1435000000,...,None,25000000,431000000,350000000,50000000,80000000,31000000,481000000,506000000,81000000
9165,2017-09-30,USD,9288000000,4894000000,1981000000,1981000000,1490000000,None,4394000000,1388000000,...,None,26000000,415000000,117000000,43000000,296000000,385000000,458000000,484000000,298000000
9166,2017-06-30,USD,7802000000,3584000000,705000000,705000000,1498000000,None,4218000000,1355000000,...,None,24000000,345000000,98000000,41000000,247000000,257000000,386000000,410000000,247000000
9167,2017-03-31,USD,7649000000,3383000000,629000000,629000000,1535000000,None,4266000000,1368000000,...,None,25000000,336000000,98000000,41000000,239000000,284000000,377000000,402000000,238000000


In [9]:
historical_data['nextQuarterPrice'] = np.nan
historical_data

,fiscalDateEnding,reportedCurrency_x,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,depreciationAndAmortization,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,nextQuarterPrice
0,2021-09-30,USD,48278000000,16426000000,4878000000,4878000000,4950000000,4916000000,31852000000,9323000000,...,131000000,1758000000,324000000,117000000,1433000000,1300000000,1875000000,2006000000,1434000000,NaN
1,2021-06-30,USD,48307000000,16343000000,4695000000,805000000,4842000000,4991000000,31995000000,9358000000,...,134000000,1939000000,415000000,121000000,1523000000,1805000000,2060000000,2194000000,1524000000,NaN
2,2021-03-31,USD,47180000000,15345000000,4636000000,501000000,4458000000,4817000000,31866000000,9240000000,...,133000000,1943000000,319000000,132000000,1626000000,1578000000,2075000000,2208000000,1624000000,NaN
3,2020-12-31,USD,47344000000,14982000000,4634000000,404000000,4239000000,4705000000,34027000000,9421000000,...,153000000,1705000000,316000000,10000000,4185000000,1688000000,1846000000,1999000000,1389000000,NaN
4,2020-09-30,USD,45390000000,14110000000,4121000000,440000000,3984000000,4623000000,31314000000,9216000000,...,137000000,1800000000,391000000,128000000,1418000000,1745000000,1928000000,2065000000,1430000000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9164,2017-12-31,USD,8586000000,4217000000,1564000000,1564000000,1427000000,1060000000,4289000000,1435000000,...,25000000,431000000,350000000,50000000,80000000,31000000,481000000,506000000,81000000,NaN
9165,2017-09-30,USD,9288000000,4894000000,1981000000,1981000000,1490000000,None,4394000000,1388000000,...,26000000,415000000,117000000,43000000,296000000,385000000,458000000,484000000,298000000,NaN
9166,2017-06-30,USD,7802000000,3584000000,705000000,705000000,1498000000,None,4218000000,1355000000,...,24000000,345000000,98000000,41000000,247000000,257000000,386000000,410000000,247000000,NaN
9167,2017-03-31,USD,7649000000,3383000000,629000000,629000000,1535000000,None,4266000000,1368000000,...,25000000,336000000,98000000,41000000,239000000,284000000,377000000,402000000,238000000,NaN


In [10]:
historical_data['currentQuarterPrice'] = np.nan
historical_data

,fiscalDateEnding,reportedCurrency_x,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,nextQuarterPrice,currentQuarterPrice
0,2021-09-30,USD,48278000000,16426000000,4878000000,4878000000,4950000000,4916000000,31852000000,9323000000,...,1758000000,324000000,117000000,1433000000,1300000000,1875000000,2006000000,1434000000,NaN,NaN
1,2021-06-30,USD,48307000000,16343000000,4695000000,805000000,4842000000,4991000000,31995000000,9358000000,...,1939000000,415000000,121000000,1523000000,1805000000,2060000000,2194000000,1524000000,NaN,NaN
2,2021-03-31,USD,47180000000,15345000000,4636000000,501000000,4458000000,4817000000,31866000000,9240000000,...,1943000000,319000000,132000000,1626000000,1578000000,2075000000,2208000000,1624000000,NaN,NaN
3,2020-12-31,USD,47344000000,14982000000,4634000000,404000000,4239000000,4705000000,34027000000,9421000000,...,1705000000,316000000,10000000,4185000000,1688000000,1846000000,1999000000,1389000000,NaN,NaN
4,2020-09-30,USD,45390000000,14110000000,4121000000,440000000,3984000000,4623000000,31314000000,9216000000,...,1800000000,391000000,128000000,1418000000,1745000000,1928000000,2065000000,1430000000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9164,2017-12-31,USD,8586000000,4217000000,1564000000,1564000000,1427000000,1060000000,4289000000,1435000000,...,431000000,350000000,50000000,80000000,31000000,481000000,506000000,81000000,NaN,NaN
9165,2017-09-30,USD,9288000000,4894000000,1981000000,1981000000,1490000000,None,4394000000,1388000000,...,415000000,117000000,43000000,296000000,385000000,458000000,484000000,298000000,NaN,NaN
9166,2017-06-30,USD,7802000000,3584000000,705000000,705000000,1498000000,None,4218000000,1355000000,...,345000000,98000000,41000000,247000000,257000000,386000000,410000000,247000000,NaN,NaN
9167,2017-03-31,USD,7649000000,3383000000,629000000,629000000,1535000000,None,4266000000,1368000000,...,336000000,98000000,41000000,239000000,284000000,377000000,402000000,238000000,NaN,NaN


In [11]:
# get all tickers and quarterly dates
tickers = historical_data['Symbol'].unique()
quarterlyDates = historical_data[['fiscalDateEnding', 'Symbol']]
quarterlyDates

,fiscalDateEnding,Symbol
0,2021-09-30,MMM
1,2021-06-30,MMM
2,2021-03-31,MMM
3,2020-12-31,MMM
4,2020-09-30,MMM
...,...,...
9164,2017-12-31,ZTS
9165,2017-09-30,ZTS
9166,2017-06-30,ZTS
9167,2017-03-31,ZTS


In [12]:
test = quarterlyDates[(quarterlyDates['fiscalDateEnding'] > '2021-06-30') & (quarterlyDates['Symbol'] == 'MMM')]
historical_data.iloc[0]['fiscalDateEnding']
monthly_prices[(monthly_prices['Date'] == '2021-09-30') & (monthly_prices['Symbol'] == "MMM")].iloc[0]['close']
historical_data.iloc[9]['nextQuarterPrice']
historical_data.at[9, 'nextQuarterPrice'] = 1
historical_data.iloc[9]['nextQuarterPrice']
len(historical_data)

9169

In [13]:
# get current quarter prices, add to dataframe
for i in range(len(historical_data)):
    print(i)
    date = historical_data.iloc[i]['fiscalDateEnding']
    sym = historical_data.iloc[i]['Symbol']
    company_price = monthly_prices[(monthly_prices['Date'] == date) & (monthly_prices['Symbol'] == sym)]
    year_month = date[:7]
    if len(company_price) == 0:
        company_price = monthly_prices[monthly_prices['Symbol'] == sym]
        company_price = company_price[company_price['Date'].str.contains(year_month, regex = False) == True]
        if len(company_price) == 0:
            month = year_month[5:]
            year = year_month[:4]
            if month[0] == '0':
                month = month[1]
            while len(company_price) == 0:
                month = int(month) + 1
                if month > 12:
                    year = int(year) + 1
                    month = 1
                if len(str(month)) == 1:
                    month = '0' + str(month)
                year_month = str(year) + '-' + str(month)
                print(year_month)
                company_price = monthly_prices[monthly_prices['Symbol'] == sym]
                company_price = company_price[company_price['Date'].str.contains(year_month, regex = False) == True]
    historical_data.at[i, 'currentQuarterPrice'] = company_price.iloc[0]['close']
    clear_output(wait = True)

9168


In [16]:
# get next quarter prices, add to dataframe
for i in range(len(historical_data)):
    print(i)
    date = historical_data.iloc[i]['fiscalDateEnding']
    #year_month = date[:7]
    sym = historical_data.iloc[i]['Symbol']
    #print(i, sym, date, year_month)
    nextQuarter = quarterlyDates[(quarterlyDates['fiscalDateEnding'] > date) & (quarterlyDates['Symbol'] == sym)]
    #print(i, nextQuarter)
    if len(nextQuarter) == 0:
        continue
    nextQuarter = nextQuarter[nextQuarter['fiscalDateEnding'] == min(nextQuarter['fiscalDateEnding'])].iloc[0]['fiscalDateEnding']
    year_month = nextQuarter[0:7]
    price = monthly_prices[(monthly_prices['Date'] == nextQuarter) & (monthly_prices['Symbol'] == sym)]
    if len(price) == 0:
        company_prices = monthly_prices[monthly_prices['Symbol'] == sym]
        price = company_prices[company_prices['Date'].str.contains(year_month, regex = False) == True]
        if len(price) == 0:
            continue
    historical_data.at[i,'nextQuarterPrice'] = price.iloc[0]['close']
    clear_output(wait = True)
historical_data

,fiscalDateEnding,reportedCurrency_x,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,incomeBeforeTax,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,nextQuarterPrice,currentQuarterPrice
0,2021-09-30,USD,48278000000,16426000000,4878000000,4878000000,4950000000,4916000000,31852000000,9323000000,...,1758000000,324000000,117000000,1433000000,1300000000,1875000000,2006000000,1434000000,NaN,175.42
1,2021-06-30,USD,48307000000,16343000000,4695000000,805000000,4842000000,4991000000,31995000000,9358000000,...,1939000000,415000000,121000000,1523000000,1805000000,2060000000,2194000000,1524000000,175.42,198.63
2,2021-03-31,USD,47180000000,15345000000,4636000000,501000000,4458000000,4817000000,31866000000,9240000000,...,1943000000,319000000,132000000,1626000000,1578000000,2075000000,2208000000,1624000000,198.63,192.68
3,2020-12-31,USD,47344000000,14982000000,4634000000,404000000,4239000000,4705000000,34027000000,9421000000,...,1705000000,316000000,10000000,4185000000,1688000000,1846000000,1999000000,1389000000,192.68,174.79
4,2020-09-30,USD,45390000000,14110000000,4121000000,440000000,3984000000,4623000000,31314000000,9216000000,...,1800000000,391000000,128000000,1418000000,1745000000,1928000000,2065000000,1430000000,174.79,160.18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9164,2017-12-31,USD,8586000000,4217000000,1564000000,1564000000,1427000000,1060000000,4289000000,1435000000,...,431000000,350000000,50000000,80000000,31000000,481000000,506000000,81000000,83.51,72.04
9165,2017-09-30,USD,9288000000,4894000000,1981000000,1981000000,1490000000,None,4394000000,1388000000,...,415000000,117000000,43000000,296000000,385000000,458000000,484000000,298000000,72.04,63.76
9166,2017-06-30,USD,7802000000,3584000000,705000000,705000000,1498000000,None,4218000000,1355000000,...,345000000,98000000,41000000,247000000,257000000,386000000,410000000,247000000,63.76,62.38
9167,2017-03-31,USD,7649000000,3383000000,629000000,629000000,1535000000,None,4266000000,1368000000,...,336000000,98000000,41000000,239000000,284000000,377000000,402000000,238000000,62.38,53.37


In [19]:
# try to fill in nan in nextQuarterPrice
for i in range(len(historical_data)):
    if math.isnan(historical_data.iloc[i]['nextQuarterPrice']):
        print(i)
        currentQuarter = historical_data.iloc[i]['fiscalDateEnding']
        #print(i, currentQuarter)
        year = currentQuarter[:4]
        month = currentQuarter[5:7]
        day = currentQuarter[8:]
        if month[0] == '0':
            month = month[1:]
            month = int(month) + 3
            if month > 12:
                month -= 12
                year = int(year) + 1
        newDate = str(year) + "-" + str(month) + "-" + str(day)
        sym = historical_data.iloc[i]['Symbol']
        nextQuarter = monthly_prices[(monthly_prices['Date'] >= newDate) & (monthly_prices['Symbol'] == sym)]
        if len(nextQuarter) == 0:
            continue
        nextQuarter = nextQuarter[nextQuarter['Date'] == min(nextQuarter['Date'])]
        historical_data.at[i,'nextQuarterPrice'] = nextQuarter.iloc[0]['close']
        clear_output(wait = True)
        #print(newDate, nextQuarter.iloc[0]['close'])

9149


In [20]:
# check for nan values in nextQuarter
count = 0
for i in range(len(historical_data)):
    if math.isnan(historical_data.iloc[i]['nextQuarterPrice']):
        #print(historical_data.iloc[i]['Symbol'])
        count += 1
count

0

In [21]:
# check for nan values in currentQuarter
count = 0
for i in range(len(historical_data)):
    if math.isnan(historical_data.iloc[i]['currentQuarterPrice']):
        #print(historical_data.iloc[i]['Symbol'])
        count += 1
count

0

In [22]:
# get percent change from current quarter to next quarter
historical_data['percentChange'] = ((historical_data['nextQuarterPrice'] - historical_data['currentQuarterPrice']) / historical_data['currentQuarterPrice']) * 100
historical_data

,fiscalDateEnding,reportedCurrency_x,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,incomeTaxExpense,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,nextQuarterPrice,currentQuarterPrice,percentChange
0,2021-09-30,USD,48278000000,16426000000,4878000000,4878000000,4950000000,4916000000,31852000000,9323000000,...,324000000,117000000,1433000000,1300000000,1875000000,2006000000,1434000000,177.63,175.42,1.259834
1,2021-06-30,USD,48307000000,16343000000,4695000000,805000000,4842000000,4991000000,31995000000,9358000000,...,415000000,121000000,1523000000,1805000000,2060000000,2194000000,1524000000,175.42,198.63,-11.685043
2,2021-03-31,USD,47180000000,15345000000,4636000000,501000000,4458000000,4817000000,31866000000,9240000000,...,319000000,132000000,1626000000,1578000000,2075000000,2208000000,1624000000,198.63,192.68,3.088022
3,2020-12-31,USD,47344000000,14982000000,4634000000,404000000,4239000000,4705000000,34027000000,9421000000,...,316000000,10000000,4185000000,1688000000,1846000000,1999000000,1389000000,192.68,174.79,10.235139
4,2020-09-30,USD,45390000000,14110000000,4121000000,440000000,3984000000,4623000000,31314000000,9216000000,...,391000000,128000000,1418000000,1745000000,1928000000,2065000000,1430000000,174.79,160.18,9.120989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9164,2017-12-31,USD,8586000000,4217000000,1564000000,1564000000,1427000000,1060000000,4289000000,1435000000,...,350000000,50000000,80000000,31000000,481000000,506000000,81000000,83.51,72.04,15.921710
9165,2017-09-30,USD,9288000000,4894000000,1981000000,1981000000,1490000000,None,4394000000,1388000000,...,117000000,43000000,296000000,385000000,458000000,484000000,298000000,72.04,63.76,12.986198
9166,2017-06-30,USD,7802000000,3584000000,705000000,705000000,1498000000,None,4218000000,1355000000,...,98000000,41000000,247000000,257000000,386000000,410000000,247000000,63.76,62.38,2.212248
9167,2017-03-31,USD,7649000000,3383000000,629000000,629000000,1535000000,None,4266000000,1368000000,...,98000000,41000000,239000000,284000000,377000000,402000000,238000000,62.38,53.37,16.882144


In [23]:
# create column for indicator of positive or negative change between quarter price
historical_data['inc_or_dec'] = [1 if x > 0 else 0 if x == 0 else -1 for x in historical_data['percentChange']]
historical_data

,fiscalDateEnding,reportedCurrency_x,totalAssets,totalCurrentAssets,cashAndCashEquivalentsAtCarryingValue,cashAndShortTermInvestments,inventory,currentNetReceivables,totalNonCurrentAssets,propertyPlantEquipment,...,interestAndDebtExpense,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,nextQuarterPrice,currentQuarterPrice,percentChange,inc_or_dec
0,2021-09-30,USD,48278000000,16426000000,4878000000,4878000000,4950000000,4916000000,31852000000,9323000000,...,117000000,1433000000,1300000000,1875000000,2006000000,1434000000,177.63,175.42,1.259834,1
1,2021-06-30,USD,48307000000,16343000000,4695000000,805000000,4842000000,4991000000,31995000000,9358000000,...,121000000,1523000000,1805000000,2060000000,2194000000,1524000000,175.42,198.63,-11.685043,-1
2,2021-03-31,USD,47180000000,15345000000,4636000000,501000000,4458000000,4817000000,31866000000,9240000000,...,132000000,1626000000,1578000000,2075000000,2208000000,1624000000,198.63,192.68,3.088022,1
3,2020-12-31,USD,47344000000,14982000000,4634000000,404000000,4239000000,4705000000,34027000000,9421000000,...,10000000,4185000000,1688000000,1846000000,1999000000,1389000000,192.68,174.79,10.235139,1
4,2020-09-30,USD,45390000000,14110000000,4121000000,440000000,3984000000,4623000000,31314000000,9216000000,...,128000000,1418000000,1745000000,1928000000,2065000000,1430000000,174.79,160.18,9.120989,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9164,2017-12-31,USD,8586000000,4217000000,1564000000,1564000000,1427000000,1060000000,4289000000,1435000000,...,50000000,80000000,31000000,481000000,506000000,81000000,83.51,72.04,15.921710,1
9165,2017-09-30,USD,9288000000,4894000000,1981000000,1981000000,1490000000,None,4394000000,1388000000,...,43000000,296000000,385000000,458000000,484000000,298000000,72.04,63.76,12.986198,1
9166,2017-06-30,USD,7802000000,3584000000,705000000,705000000,1498000000,None,4218000000,1355000000,...,41000000,247000000,257000000,386000000,410000000,247000000,63.76,62.38,2.212248,1
9167,2017-03-31,USD,7649000000,3383000000,629000000,629000000,1535000000,None,4266000000,1368000000,...,41000000,239000000,284000000,377000000,402000000,238000000,62.38,53.37,16.882144,1


In [25]:
# get non numeric columns
non_numeric_columns = []
for i in historical_data.columns:
    try:
        float(historical_data[i].iloc[0])
    except:
        non_numeric_columns.append(i)
non_numeric_columns

['fiscalDateEnding',
 'reportedCurrency_x',
 'longTermInvestments',
 'Symbol',
 'reportedCurrency_y',
 'paymentsForOperatingActivities',
 'proceedsFromOperatingActivities',
 'paymentsForRepurchaseOfPreferredStock',
 'dividendPayoutPreferredStock',
 'proceedsFromIssuanceOfCommonStock',
 'proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet',
 'proceedsFromIssuanceOfPreferredStock',
 'changeInExchangeRate',
 'reportedDate',
 'reportedCurrency',
 'investmentIncomeNet',
 'depreciation']

In [26]:
def replaceNone(df):
    return df.replace("None", np.nan)

In [27]:
# test linear regression model on data
#mmm = historical_data[historical_data['Symbol'] == 'MMM']
#mmm = mmm.apply(replanceNone)
#mmm = mmm.dropna()
#X = mmm.drop(non_numeric_columns, axis = 1)
#X = X.drop(['nextQuarterPrice'], axis = 1)
#Y = mmm['nextQuarterPrice']
#X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .3)
#model = LinearRegression()
#model.fit(X_train, y_train)
#print(y_test)
#model.predict(X_test)
#for i in X.columns:
#    pd.to_numeric(X[i])

In [28]:
correlation = historical_data.drop(non_numeric_columns, axis = 1)
correlation = correlation.replace('None', np.NaN)
correlation = correlation.astype('float64')
imp = SimpleImputer(missing_values = np.NaN, strategy='mean')
newDf = pd.DataFrame(imp.fit_transform(correlation))
newDf.columns = correlation.columns
newDf.index = correlation.index
correlation = newDf
correlation = (correlation - correlation.mean()) / correlation.std()
correlation = correlation.corr()['nextQuarterPrice'].drop(['nextQuarterPrice'])
correlation = correlation[correlation > .1].index
correlation
len(correlation)

28

In [29]:
# Linear Regression
df = historical_data
X = df.drop(non_numeric_columns, axis = 1)
X = X.drop(['nextQuarterPrice', 'percentChange', 'inc_or_dec'], axis = 1)
X = X.replace('None', np.NaN)
X = X.astype('float64')
X = (X - X.mean()) / X.std()
imp = SimpleImputer(missing_values = np.NaN, strategy='mean')
newDf = pd.DataFrame(imp.fit_transform(X))
newDf.columns = X.columns
newDf.index = X.index
X = newDf
X = X[correlation]
#column_means = X.mean()
#X = X.fillna(column_means)
Y = df['nextQuarterPrice']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .25)
model = LinearRegression()
model.fit(X_train, y_train)
#print(y_test)
model.predict(X_test)
X_test

,currentNetReceivables,currentAccountsPayable,capitalLeaseObligations,totalShareholderEquity,commonStock,operatingCashflow,depreciationDepletionAndAmortization,capitalExpenditures,profitLoss,paymentsForRepurchaseOfCommonStock,...,operatingExpenses,nonInterestIncome,depreciationAndAmortization,incomeBeforeTax,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,currentQuarterPrice
6651,-2.644700e-17,-3.516373e-01,-2.757194e-01,-0.367500,-0.204110,-0.169386,-0.302339,-0.352585,-0.202499,-2.653488e-01,...,-0.410764,-0.263790,-0.319764,-0.225618,-0.210384,-0.177080,-0.197393,-0.240222,-0.218628,-0.049026
6986,-2.644700e-17,-2.146006e-17,-2.572194e-17,-0.214354,-0.214862,-0.078611,-0.183715,1.916081,-0.119118,-1.355308e-18,...,-0.354549,-0.348552,-0.111375,-0.150020,-0.184216,-0.098920,-0.206940,-0.217270,-0.103004,0.667411
1496,-4.632328e-01,-2.146006e-17,-2.572194e-17,-0.422355,-0.000684,-0.191862,-0.337578,-0.362072,-0.265484,-2.293729e-01,...,-0.306489,-0.301984,-0.396218,-0.282594,-0.239648,-0.253040,-0.234813,-0.288943,-0.256908,-0.376981
8825,-2.912493e-01,-2.416754e-01,-2.572194e-17,-0.184465,-0.216489,-0.119361,-0.135329,-1.785309,-0.215410,-2.435255e-01,...,-0.185503,0.000000,-0.035691,-0.196111,-0.199587,-0.200708,-0.185713,-0.172388,-0.162677,-0.279073
6044,-4.666782e-01,-3.070093e-01,-2.448072e-02,0.148581,-0.069912,-0.008909,0.238334,-0.048369,0.074033,-3.380852e-02,...,-0.524407,-0.122179,0.548778,0.014446,0.039196,0.101771,0.494603,0.060595,0.100600,-0.377144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5680,-4.231613e-01,-3.711124e-01,-2.572194e-17,-0.443039,-0.216802,-0.183910,-0.366435,-0.335736,-0.262846,-2.040870e-01,...,-0.339945,-0.348904,-0.384024,-0.265578,-0.237519,-0.236101,-0.223368,-0.275778,-0.254122,1.998491
4662,-4.256693e-01,-3.832262e-01,-2.572194e-17,-0.319538,-0.216842,-0.137833,-0.336955,-0.321795,-0.172120,-2.654643e-01,...,-0.329640,-0.277723,-0.396841,-0.201122,-0.164735,-0.153455,-0.188154,-0.243090,-0.158324,3.715028
9147,-2.644700e-17,-2.146006e-17,-2.572194e-17,-0.242516,0.318060,-0.132052,-0.367403,-0.323179,-0.246893,-2.293808e-01,...,-0.311465,-0.337272,-0.339791,-0.246733,-0.516085,-0.224525,-0.206885,-0.252679,-0.237277,-0.390559
443,2.072734e+00,7.798768e-02,-2.572194e-17,3.907682,4.294945,1.970244,1.525123,3.124803,2.827436,-2.654643e-01,...,2.118765,-0.353574,2.561522,3.063664,2.255782,3.378553,1.861496,2.234697,3.024257,3.360163


In [30]:
historical_data.loc[X_test.index, ['Symbol', 'nextQuarterPrice']]
predicted = model.predict(X_test)
predicted
new_df = historical_data.loc[X_test.index, ['Symbol', 'nextQuarterPrice']]
new_df['predicted'] = predicted
new_df['percentage_diff'] = (abs(new_df['predicted'] - new_df['nextQuarterPrice']) / ((new_df['predicted'] + new_df['nextQuarterPrice']) / 2)) * 100
#new_df[abs(new_df['predicted'] - new_df['nextQuarterPrice']) < 8]
new_df[new_df['percentage_diff'] < 5]
new_df

,Symbol,nextQuarterPrice,predicted,percentage_diff
6651,PKI,128.29,130.699020,1.860326
6986,PSA,297.10,310.722556,4.482412
1496,CDNS,43.48,46.351496,6.393071
8825,WEC,79.08,74.518415,5.939626
6044,NEM,61.74,46.714246,27.708927
...,...,...,...,...
5680,MTD,619.52,655.906697,5.705808
4662,ISRG,364.94,1095.972198,100.078868
9147,ZION,43.91,43.840761,0.157808
443,GOOG,1046.40,1018.187817,2.732960


In [31]:
# neural network predicting if price will increase next quarter or not
df = historical_data
X = df.drop(non_numeric_columns, axis = 1)
X = X.drop(['nextQuarterPrice', 'percentChange', 'inc_or_dec'], axis = 1)
X = X.replace('None', np.NaN)
X = X.astype('float64')
X = (X - X.mean()) / X.std()
imp = SimpleImputer(missing_values = np.NaN, strategy='mean')
newDf = pd.DataFrame(imp.fit_transform(X))
newDf.columns = X.columns
newDf.index = X.index
X = newDf
X = X[correlation]
Y = df['inc_or_dec']
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
Y = np_utils.to_categorical(encoded_Y)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = .25)
model = models.Sequential()
model.add(layers.Dense(18, activation='relu', input_shape=(28, )))
model.add(layers.Dense(18, activation='relu'))
model.add(layers.Dense(9, activation='relu'))
model.add(layers.Dense(3, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs = 7, batch_size = 300)

2022-02-01 21:13:03.527205: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/7
23/23 [==============================] - 2s 1ms/step - loss: 1.0833 - accuracy: 0.5726
Epoch 2/7
23/23 [==============================] - 0s 939us/step - loss: 0.9135 - accuracy: 0.6446
Epoch 3/7
23/23 [==============================] - 0s 927us/step - loss: 0.8158 - accuracy: 0.6478
Epoch 4/7
23/23 [==============================] - 0s 957us/step - loss: 0.7482 - accuracy: 0.6472
Epoch 5/7
23/23 [==============================] - 0s 1000us/step - loss: 0.7071 - accuracy: 0.6489
Epoch 6/7
23/23 [==============================] - 0s 1ms/step - loss: 0.6830 - accuracy: 0.6489
Epoch 7/7
23/23 [==============================] - 0s 972us/step - loss: 0.6706 - accuracy: 0.6499


In [32]:
# get accuracy of neural network
model.evaluate(X_test, y_test)

72/72 [==============================] - 0s 600us/step - loss: 0.6827 - accuracy: 0.6324


[0.6827347874641418, 0.6323593258857727]

In [34]:
X_test

,currentNetReceivables,currentAccountsPayable,capitalLeaseObligations,totalShareholderEquity,commonStock,operatingCashflow,depreciationDepletionAndAmortization,capitalExpenditures,profitLoss,paymentsForRepurchaseOfCommonStock,...,operatingExpenses,nonInterestIncome,depreciationAndAmortization,incomeBeforeTax,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,currentQuarterPrice
4771,1.842842e-02,-2.109425e-01,-2.572194e-17,-0.295145,-0.202025,-0.136591,-0.344664,-0.345437,-0.192992,-2.654643e-01,...,-0.287523,-0.061638,-3.645696e-01,-0.192118,-0.191427,-0.162813,-0.211031,-0.224964,-0.184875,-0.215519
3205,-2.644700e-17,-2.146006e-17,-3.178971e-01,-0.286279,-0.216853,-0.177661,-0.241022,-0.370034,-0.266073,-2.654643e-01,...,-0.407933,-0.317008,-2.010106e-01,-0.283230,-0.242310,-0.254011,-0.229554,-0.253456,-0.260392,0.631585
7168,-2.644700e-17,-2.146006e-17,-2.972940e-01,-0.169874,1.344161,-0.160971,-0.188449,-0.215725,-0.261384,-1.355308e-18,...,-0.402491,-0.348904,-1.187793e-01,-0.277878,-0.145950,-0.249623,-0.212589,-0.225203,-0.252700,-0.318989
2237,-2.644700e-17,-2.146006e-17,-2.572194e-17,-0.235431,-0.086884,-0.161731,-0.368479,-0.371617,-0.216909,-1.651523e-01,...,-0.330927,-0.324583,-3.532538e-01,-0.196621,-0.481309,-0.190705,-0.174783,-0.223154,-0.205617,-0.262796
8537,3.167077e+00,-2.146006e-17,4.132188e-02,1.669737,-0.167987,2.158946,0.163021,6.442299,2.758566,-1.355308e-18,...,2.617633,-0.348904,6.263893e+00,3.168985,2.200216,2.529705,2.132646,3.076538,2.863481,-0.341441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5378,-2.236010e-01,-1.640358e-01,-2.572194e-17,-0.130127,-0.110121,-0.175020,0.320103,0.151314,-0.525185,-2.654643e-01,...,-0.247926,-0.351283,-2.424509e-17,-0.519379,-0.503611,-0.581308,-0.361298,-0.436880,-0.602363,-0.506870
4861,6.222083e-01,3.619785e-01,-2.572194e-17,0.110822,-0.215828,0.154544,-0.369555,-0.605898,0.122289,-1.030887e-01,...,0.118712,-0.341766,-2.236708e-01,-0.058599,-0.317636,0.242770,-0.068923,-0.098785,0.126819,-0.397552
8964,-5.476989e-02,3.877635e-01,-2.572194e-17,-0.324675,-0.204210,0.088986,-0.198485,-0.076047,-0.224874,-1.932974e-01,...,-0.020959,0.000000,-1.344773e-01,-0.196621,-0.144512,-0.317889,-0.171068,-0.185530,-0.216995,0.181061
8775,-6.568356e-02,-1.498096e-01,-2.572194e-17,-0.253019,-0.216170,0.014790,0.168400,0.018851,-0.147570,-8.504702e-02,...,-0.283416,0.026121,4.393902e-01,-0.131220,-0.144512,-0.118300,-0.052738,-0.041827,-0.132402,0.010684


In [38]:
print(historical_data.iloc[3205])
historical_data.iloc[3205]['Symbol']

fiscalDateEnding                          2019-06-30
reportedCurrency_x                               USD
totalAssets                              12671145000
totalCurrentAssets                          54971000
cashAndCashEquivalentsAtCarryingValue       38168000
                                            ...     
netIncome_y                                 92275000
nextQuarterPrice                              326.65
currentQuarterPrice                           291.93
percentChange                                11.8933
inc_or_dec                                         1
Name: 3205, Length: 101, dtype: object


'ESS'

In [42]:
# find which encoded value is predicted
predicted = model.predict(X_test)
predicted.argmax(axis = 1)
predicted

array([[0.33749676, 0.00542051, 0.65708274],
       [0.31440136, 0.00100876, 0.6845899 ],
       [0.3503964 , 0.00265481, 0.64694875],
       ...,
       [0.37082025, 0.01470664, 0.61447316],
       [0.3737615 , 0.01608269, 0.61015576],
       [0.36057684, 0.07386205, 0.5655611 ]], dtype=float32)

In [50]:
# find which X_test rows were predicted correctly
y_test.argmax(axis = 1)
nn = pd.DataFrame({'predicted': predicted.argmax(axis = 1), 'actual': y_test.argmax(axis = 1)})
X_test.iloc[nn[nn['predicted'] == nn['actual']].index]

,currentNetReceivables,currentAccountsPayable,capitalLeaseObligations,totalShareholderEquity,commonStock,operatingCashflow,depreciationDepletionAndAmortization,capitalExpenditures,profitLoss,paymentsForRepurchaseOfCommonStock,...,operatingExpenses,nonInterestIncome,depreciationAndAmortization,incomeBeforeTax,netIncomeFromContinuingOperations,comprehensiveIncomeNetOfTax,ebit,ebitda,netIncome_y,currentQuarterPrice
4771,1.842842e-02,-2.109425e-01,-2.572194e-17,-0.295145,-0.202025,-0.136591,-0.344664,-0.345437,-0.192992,-2.654643e-01,...,-0.287523,-0.061638,-3.645696e-01,-0.192118,-0.191427,-0.162813,-0.211031,-0.224964,-0.184875,-0.215519
3205,-2.644700e-17,-2.146006e-17,-3.178971e-01,-0.286279,-0.216853,-0.177661,-0.241022,-0.370034,-0.266073,-2.654643e-01,...,-0.407933,-0.317008,-2.010106e-01,-0.283230,-0.242310,-0.254011,-0.229554,-0.253456,-0.260392,0.631585
7168,-2.644700e-17,-2.146006e-17,-2.972940e-01,-0.169874,1.344161,-0.160971,-0.188449,-0.215725,-0.261384,-1.355308e-18,...,-0.402491,-0.348904,-1.187793e-01,-0.277878,-0.145950,-0.249623,-0.212589,-0.225203,-0.252700,-0.318989
2237,-2.644700e-17,-2.146006e-17,-2.572194e-17,-0.235431,-0.086884,-0.161731,-0.368479,-0.371617,-0.216909,-1.651523e-01,...,-0.330927,-0.324583,-3.532538e-01,-0.196621,-0.481309,-0.190705,-0.174783,-0.223154,-0.205617,-0.262796
3088,-2.949589e-01,-1.379961e-01,-3.425571e-01,-0.234390,-0.216563,-0.100485,0.007535,0.570390,-0.195423,-2.654643e-01,...,0.325693,-0.113741,1.877720e-01,-0.339813,-0.183121,-0.203911,-0.224802,-0.188416,-0.184631,-0.231918
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2961,-1.094871e-01,-2.041004e-01,-1.603285e-01,-0.294607,-0.175619,-0.130546,-0.314899,-0.283737,-1.263033,-2.584641e-01,...,-0.171679,-0.112249,-3.165667e-01,-1.180949,-0.228466,-1.194699,-0.211661,-0.814481,-1.310225,0.251323
6176,-3.684883e-01,-3.154383e-01,-2.572194e-17,-0.340843,-0.216478,-0.125164,-0.215377,-0.064185,-0.213020,-2.622889e-01,...,-0.073279,-0.348904,-1.608988e-01,-0.183668,-0.197319,-0.192706,-0.151780,-0.170637,-0.201511,-0.465032
5378,-2.236010e-01,-1.640358e-01,-2.572194e-17,-0.130127,-0.110121,-0.175020,0.320103,0.151314,-0.525185,-2.654643e-01,...,-0.247926,-0.351283,-2.424509e-17,-0.519379,-0.503611,-0.581308,-0.361298,-0.436880,-0.602363,-0.506870
8775,-6.568356e-02,-1.498096e-01,-2.572194e-17,-0.253019,-0.216170,0.014790,0.168400,0.018851,-0.147570,-8.504702e-02,...,-0.283416,0.026121,4.393902e-01,-0.131220,-0.144512,-0.118300,-0.052738,-0.041827,-0.132402,0.010684


In [45]:
# look at actual values of predicted
Y

array([[0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [1., 0., 0.]], dtype=float32)

In [66]:
# Random forest
forest = RandomForestClassifier(n_estimators = 25, criterion = 'entropy')
forest.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=25)

In [67]:
# prediction of random forest
predictions = forest.predict(X_test)
total = 0
for x in range(len(predictions)):
    if np.array_equal(predictions[x], y_test[x]):
        total += 1
total / len(predictions)

0.6088094199738334

In [51]:
test = pd.DataFrame({'A': [1, 2, -1, 0, .1], 'percentChange': [3, -1, 5, 0, .1]})
test['inc_or_dec'] = [1 if x > 0 else 0 if x == 0 else -1 for x in test['percentChange']]
test

,A,percentChange,inc_or_dec
0,1.0,3.0,1
1,2.0,-1.0,-1
2,-1.0,5.0,1
3,0.0,0.0,0
4,0.1,0.1,1


In [52]:
historical_data.to_csv("historical_data.csv")

In [9]:
historical_data = historical_data.set_index(['Symbol', 'fiscalDateEnding'])
historical_data

reportedCurrency_x  totalAssets totalCurrentAssets  \
Symbol fiscalDateEnding                                                      
MMM    2021-09-30                      USD  48278000000        16426000000   
       2021-06-30                      USD  48307000000        16343000000   
       2021-03-31                      USD  47180000000        15345000000   
       2020-12-31                      USD  47344000000        14982000000   
       2020-09-30                      USD  45390000000        14110000000   
...                                    ...          ...                ...   
ZTS    2017-12-31                      USD   8586000000         4217000000   
       2017-09-30                      USD   9288000000         4894000000   
       2017-06-30                      USD   7802000000         3584000000   
       2017-03-31                      USD   7649000000         3383000000   
       2016-12-31                      USD   7649000000         3390000000   

                        cashAndCashEquivalentsAtCarryingValue  \
Symbol fiscalDateEnding                                         
MMM    2021-09-30                                  4878000000   
       2021-06-30                                  4695000000   
       2021-03-31                                  4636000000   
       2020-12-31                                  4634000000   
       2020-09-30                                  4121000000   
...                                                       ...   
ZTS    2017-12-31                                  1564000000   
       2017-09-30                                  1981000000   
       2017-06-30                                   705000000   
       2017-03-31                                   629000000   
       2016-12-31                                   727000000   

                        cashAndShortTermInvestments   inventory  \
Symbol fiscalDateEnding                                           
MMM    2021-09-30                        4878000000  4950000000   
       2021-06-30                         805000000  4842000000   
       2021-03-31                         501000000  4458000000   
       2020-12-31                         404000000  4239000000   
       2020-09-30                         440000000  3984000000   
...                                             ...         ...   
ZTS    2017-12-31                        1564000000  1427000000   
       2017-09-30                        1981000000  1490000000   
       2017-06-30                         705000000  1498000000   
       2017-03-31                         629000000  1535000000   
       2016-12-31                         727000000  1502000000   

                        currentNetReceivables totalNonCurrentAssets  \
Symbol fiscalDateEnding                                               
MMM    2021-09-30                  4916000000           31852000000   
       2021-06-30                  4991000000           31995000000   
       2021-03-31                  4817000000           31866000000   
       2020-12-31                  4705000000           34027000000   
       2020-09-30                  4623000000           31314000000   
...                                       ...                   ...   
ZTS    2017-12-31                  1060000000            4289000000   
       2017-09-30                        None            4394000000   
       2017-06-30                        None            4218000000   
       2017-03-31                        None            4266000000   
       2016-12-31                   962000000            4163000000   

                        propertyPlantEquipment  \
Symbol fiscalDateEnding                          
MMM    2021-09-30                   9323000000   
       2021-06-30                   9358000000   
       2021-03-31                   9240000000   
       2020-12-31                   9421000000   
       2020-09-30                   9216000000   
...              

In [24]:
monthly_prices[monthly_prices['Symbol'] == 'MMM']

,Date,open,high,low,close,volume,Symbol
0,2022-01-21,178.32,181.780,172.5600,172.65,34039389,MMM
1,2021-12-31,172.55,179.180,169.1801,177.63,48251474,MMM
2,2021-11-30,178.53,186.300,169.9400,170.04,47872133,MMM
3,2021-10-29,176.25,183.820,173.7800,178.68,48241875,MMM
4,2021-09-30,194.55,195.606,175.3700,175.42,67366254,MMM
...,...,...,...,...,...,...,...
261,2000-04-28,89.38,98.310,85.3100,86.63,34480000,MMM
262,2000-03-31,87.19,94.630,78.1900,88.56,43561100,MMM
263,2000-02-29,94.38,96.190,81.3100,88.19,33435200,MMM
264,2000-01-31,96.06,103.800,90.2500,93.63,34253500,MMM
